In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-depression-machine-learning-challenge/sample_submission.csv
/kaggle/input/predicting-depression-machine-learning-challenge/train.csv
/kaggle/input/predicting-depression-machine-learning-challenge/test.csv


In [60]:
train = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/train.csv')
test = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/test.csv')

submission = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/sample_submission.csv')

In [61]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [43]:
train["is_profession_missing"] = train["Profession"].notna().astype(int)
train["is_Work_Pressure_missing"] = train["Work Pressure"].notna().astype(int)
train["is_Job_Satisfaction_missing"] = train["Job Satisfaction"].notna().astype(int)

test["is_profession_missing"] = test["Profession"].notna().astype(int)
test["is_Work_Pressure_missing"] = test["Work Pressure"].notna().astype(int)
test["is_Job_Satisfaction_missing"] = test["Job Satisfaction"].notna().astype(int)

In [62]:
# Define target variable (change 'target_column' to the actual target name)
target = "Depression"

# Define feature columns (exclude target column)
features = [col for col in train.columns if col != target]


In [63]:
# Identify categorical columns
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

# Convert categorical columns to string (CatBoost prefers them as strings)
train[categorical_features] = train[categorical_features].astype(str)


In [64]:
X_train, X_test, y_train, y_test = train_test_split(
    train[features], train[target], test_size=0.2, random_state=42
)


In [65]:
train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)
test_pool = Pool(X_test, label=y_test, cat_features=categorical_features)


In [66]:
model = CatBoostClassifier(iterations=500, learning_rate=0.05, max_depth=8, verbose=100)
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)


0:	learn: 0.6067950	test: 0.6068895	best: 0.6068895 (0)	total: 162ms	remaining: 1m 20s
100:	learn: 0.1469184	test: 0.1501691	best: 0.1501691 (100)	total: 14.6s	remaining: 57.5s
200:	learn: 0.1404546	test: 0.1482742	best: 0.1482742 (200)	total: 26.9s	remaining: 40.1s
300:	learn: 0.1356341	test: 0.1476251	best: 0.1476184 (292)	total: 39.7s	remaining: 26.3s
400:	learn: 0.1316293	test: 0.1473426	best: 0.1473426 (400)	total: 52.7s	remaining: 13s
499:	learn: 0.1277288	test: 0.1473153	best: 0.1472724 (462)	total: 1m 5s	remaining: 0us

bestTest = 0.1472724117
bestIteration = 462

Shrink model to first 463 iterations.


In [67]:
y_pred = model.predict(X_test)


In [68]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9395


In [69]:
# Identify categorical columns (if applicable)
categorical_features_test = test.select_dtypes(include=['object']).columns.tolist()

# Convert categorical columns to string (CatBoost prefers categorical variables as strings)
test[categorical_features_test] = test[categorical_features_test].astype(str)


In [70]:
# Make predictions
predictions = model.predict(test)


In [71]:
predictions

array([0, 0, 0, ..., 0, 1, 0])

In [72]:
submission

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,0
4,140704,0
...,...,...
93795,234495,0
93796,234496,0
93797,234497,0
93798,234498,0


In [73]:
submission["Depression"] = predictions

In [74]:
submission

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1


In [75]:
submission["Depression"].value_counts()

Depression
0    77041
1    16759
Name: count, dtype: int64

In [76]:
submission.to_csv("Submission_15.csv",index=False)